In [50]:
import numpy as np 
import pandas as pd
import tensorflow as tf 
import keras
from tensorflow import image
from keras.models import Sequential 
from keras import layers
from keras.layers import Conv2D, Dense, Activation, Dropout, Flatten
import pathlib 
import os
import shutil
from shutil import move

In [51]:
data_dir = '/Users/William/Desktop/Jupyter/HAM10000/'
img_height = 450
img_width = 600
batch_size = 32
input_shape = (450, 600, 3) #w, h, c? 
epochs = 50
#train_ds = tf.keras.preprocessing.image_dataset_from_directory(directory)

In [28]:
os.chdir(data_dir)
df = pd.read_csv(data_dir + '/HAM10000_metadata.csv')
all_images = os.listdir(data_dir + '/HAM10000')

for image in all_images:
    skinType = df[df['image_id'] + '.jpg' == image]['dx']
    skinType = str(list(skinType)[0])
    if not os.path.exists(os.path.join(data_dir, skinType)):
        os.mkdir(os.path.join(data_dir, skinType))
    path_from = os.path.join(data_dir + '/HAM10000', image)
    path_to = os.path.join(data_dir, skinType, image)
    move(path_from, path_to)

shutil.rmtree(data_dir + '/HAM10000')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/William/Desktop/Jupyter/HAM10000//HAM10000'

In [52]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, 
    label_mode = 'categorical',
    validation_split = .2, 
    subset = "training", 
    seed = 123, 
    image_size = (img_height, img_width), 
    batch_size = batch_size
)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.


In [53]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    label_mode = 'categorical',
    validation_split = .2, 
    subset = "validation", 
    seed = 123, 
    image_size = (img_height, img_width), 
    batch_size = batch_size
)
print(test_ds.class_names)

Found 10015 files belonging to 7 classes.
Using 2003 files for validation.
['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


In [54]:
num_classes = 7

model = Sequential()
model.add(layers.experimental.preprocessing.Resizing(150, 200, interpolation = 'bilinear'))
model.add(layers.experimental.preprocessing.Resizing(75, 100, interpolation = 'bilinear'))
model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape = input_shape))
model.add(layers.Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, (3,3), padding = 'same', activation = 'relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [55]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = .0005), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
#model.summary()

In [56]:
#fitting our model to the training and validation dataset
classifier = model.fit(
    train_ds, 
    batch_size = batch_size, 
    epochs = epochs
)

Epoch 1/50
251/251 [==============================] - 67s 267ms/step - loss: 1.0635 - accuracy: 0.6651
Epoch 2/50
251/251 [==============================] - 67s 268ms/step - loss: 0.9247 - accuracy: 0.6755
Epoch 3/50
251/251 [==============================] - 67s 268ms/step - loss: 0.8832 - accuracy: 0.6838
Epoch 4/50
251/251 [==============================] - 67s 268ms/step - loss: 0.8472 - accuracy: 0.6862
Epoch 5/50
251/251 [==============================] - 68s 269ms/step - loss: 0.8107 - accuracy: 0.6966
Epoch 6/50
251/251 [==============================] - 68s 271ms/step - loss: 0.7814 - accuracy: 0.7066
Epoch 7/50
251/251 [==============================] - 68s 270ms/step - loss: 0.7779 - accuracy: 0.7184
Epoch 8/50
251/251 [==============================] - 68s 272ms/step - loss: 0.7452 - accuracy: 0.7238
Epoch 9/50
251/251 [==============================] - 68s 272ms/step - loss: 0.7322 - accuracy: 0.7245
Epoch 10/50
251/251 [==============================] - 69s 274ms/step - l

In [57]:
#evaluating our model on the test dataset
evaluation = model.evaluate(test_ds)

63/63 [==============================] - 24s 377ms/step - loss: 1.0035 - accuracy: 0.7594


In [ ]:
tf.math.confusion_matrix()